# TP1 Sandbox
- This notebook can be used as a sandbox for you to explore the data and identify formating and sanitizing problems.
- You may also use it to compose the sample dataframes and test if the cleaning function output matches the expected result.
- As you validate your methods, create the corresponding function in `loader.py`
- Once your script is ready, you can import the loading function and use it to load a clean dataframe.

# Exploration space
## Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 51)
pd.set_option('display.width', 1000)

## (Down)Load data

In [3]:

from loader import download_data
url_mtp = "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_DAE.csv"


data_path = download_data(url_mtp)
data = pd.read_csv(data_path)

## Drop empty columns

In [4]:
data.dropna(
    axis='columns',
    how='all',
    inplace=True)
data

,nom,lat_coor1,x,long_coor1,y,adr_num,adr_voie,com_cp,com_insee,com_nom,acc,acc_lib,acc_pcsec,acc_acc,acc_etg,acc_complt,photo1,photo2,disp_j,disp_h,disp_compl,tel1,tel2,site_email,date_insta,etat_fonct,fab_siren,fab_rais,mnt_siren,mnt_rais,modele,num_serie,id_euro,lc_ped,dtpr_lcped,dtpr_lcad,dtpr_bat,freq_mnt,dispsurv,dermnt,expt_siren,expt_rais,expt_tel1,expt_tel2,expt_email,ref,id,appartenan,dae_mobile
0,"Plateau sportif de GrammontTerrain 9, 10, 11",3.93392108647369,775412.3055,43.6136351580956,6.279844e+06,,avenue albert Einstein,34000,34172,Montpellier,intérieur,non,non,oui,0,,,,"lundi, mardi, mercredi, jeudi, vendredi",Heures ouvrables,-,334 67 64 87 70,,,2017-12-1,En fonctionnement,,ZOLL,,,AED PLUS,X17I955734,,oui,2021-02-02,2023-01-14,2022-12-1,tous les ans,,2019-05-15,213401722,personne morale,334 67 34 70 00,,mairie@ville-montpellier.fr,GARCIA Serge,22,Mairie,non
1,MEDIATHEQUE JEAN-JACQUES ROUSSEAU,,0.0000,,0.000000e+00,,,0,0,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,Métropole,
2,MEDIATHEQUE William SHAKESPEARE,,0.0000,,0.000000e+00,,,0,0,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,Métropole,
3,MIBI,,0.0000,,0.000000e+00,,,0,0,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,Métropole,
4,MAISON POUR TOUS LEO LAGRANGE,,0.0000,,0.000000e+00,155,Rue de Bologne,34172,34080,Montpellier,intérieur,oui,non,oui,0,,,,"lundi, mardi, mercredi, jeudi, vendredi",Heures ouvrables,,+334 67 40 33 57,,,2020-10-06,En fonctionnement,,ZOLL,,,AED PLUS,X19J215834,,,2021-08-17,2024-12-28,2025-10-06,Tous les ans,,2020-10-06,213401722,personne morale,+334 67 34 70 00,,mairie@ville-montpellier.fr,,0,Mairie,non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Zoo de Lunaret,3.87346840734586,770497.5034,43.6409437259243,6.282823e+06,,avenue agropolis,34090,34172,Montpellier,intérieur,non,non,non,0,Réfectoire personnel,,,"mardi, mercredi, jeudi, vendredi, samedi, dima...",Heures ouvrables,,334 67 54 45 20,,,2013-09-05,En fonctionnement,,ZOLL,,,AED PLUS,X13G616857,,,2020-10-01,2023-09-09,2022-12-01,Tous les ans,,2019-05-14,213401722,personne morale,334 67 34 70 00,,mairie@ville-montpellier.fr,CORNIER,0,Mairie,non
271,Gymnase Jean Bouin,3.82081921967322,766257.7753,43.6327880433471,6.281871e+06,-,Avenue du Biterrois,34080,34172,Montpellier,interieur,non,non,non,0,,,,"lundi, mardi, mercredi, jeudi, vendredi",Heures ouvrables,,334 67 75 44 43,,,2019-11-01,En fonctionnement,,ZOLL,,,AED PLUS,X19J212182,,,2021-10-26,2024-11-03,2024-11-01,Tous les ans,,2019-11-01,213401722,personne morale,334 67 34 70 00,,mairie@ville-montpellier.fr,,0,Mairie,non
272,Centre Culturel Rabelais,3.88032026773112,771087.9776,43.6106902122358,6.279467e+06,29,boulevard Sarrail,34000,34172,Montpellier,intérieur,non,non,oui,0,,,,"lundi, mardi, mercredi, jeudi, vendredi",Heures ouvrables,,334 67 34 71 33,,,2019-12-01,En fonctionnement,,ZOLL,,,AED PLUS,X19J215337,,,2021-08-17,2024-12-28,2024-12-01,2019-12-01,,Tous les ans,213401722,personne morale,334 67 34 70 00,,mairie@ville-montpellier.fr,,76,Mairie,non
273,,3.86982233698134,770237.7162,43.6127835439949,6.279690e+06,"1 place Jacques Mirouse, MONTPELLIER","1 place Jacques Mirouse, MONTPELLIER",34000,34172,Montpellier,intérieur,non,non,non,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,


## Keeping only columns of interest

We want to build a new dataframe containing:
- ID (the dataframe's index)
- Name
- Adress (including postal code (com_cp) and city name (com_nom))
- Contact phone number
- Maintenance frequency
- Latest maintenance date
- Longitude
- Latitude

In [5]:
kept_columns = [
    'nom', 'adr_num','adr_voie',
    'com_cp', 'com_nom', 
    'tel1',
    'freq_mnt', 'dermnt',
    'lat_coor1', 'long_coor1']
data_filter = data.filter(items=kept_columns)
data_filter


,nom,adr_num,adr_voie,com_cp,com_nom,tel1,freq_mnt,dermnt,lat_coor1,long_coor1
0,"Plateau sportif de GrammontTerrain 9, 10, 11",,avenue albert Einstein,34000,Montpellier,334 67 64 87 70,tous les ans,2019-05-15,3.93392108647369,43.6136351580956
1,MEDIATHEQUE JEAN-JACQUES ROUSSEAU,,,0,,,,,,
2,MEDIATHEQUE William SHAKESPEARE,,,0,,,,,,
3,MIBI,,,0,,,,,,
4,MAISON POUR TOUS LEO LAGRANGE,155,Rue de Bologne,34172,Montpellier,+334 67 40 33 57,Tous les ans,2020-10-06,,
...,...,...,...,...,...,...,...,...,...,...
270,Zoo de Lunaret,,avenue agropolis,34090,Montpellier,334 67 54 45 20,Tous les ans,2019-05-14,3.87346840734586,43.6409437259243
271,Gymnase Jean Bouin,-,Avenue du Biterrois,34080,Montpellier,334 67 75 44 43,Tous les ans,2019-11-01,3.82081921967322,43.6327880433471
272,Centre Culturel Rabelais,29,boulevard Sarrail,34000,Montpellier,334 67 34 71 33,2019-12-01,Tous les ans,3.88032026773112,43.6106902122358
273,,"1 place Jacques Mirouse, MONTPELLIER","1 place Jacques Mirouse, MONTPELLIER",34000,Montpellier,,,,3.86982233698134,43.6127835439949


In [6]:
data_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   nom         275 non-null    object
 1   adr_num     275 non-null    object
 2   adr_voie    275 non-null    object
 3   com_cp      275 non-null    int64 
 4   com_nom     275 non-null    object
 5   tel1        275 non-null    object
 6   freq_mnt    275 non-null    object
 7   dermnt      275 non-null    object
 8   lat_coor1   275 non-null    object
 9   long_coor1  275 non-null    object
dtypes: int64(1), object(9)
memory usage: 21.6+ KB


## Extract problematic cases to specify cleaning functions

Use this space to explore the dataframe and identify problems in formatting and sanitizing. Save the indexes of interesting problematic cases in the following set `idx_problem_cases`, as you expore the data.

For instance, if you identify examples 3 and 45 to be dirty, add them to the set with:
`idx_problem_cases.update([3, 45])`

In [7]:
idx_problem_cases = set()

### Address data

In [8]:
data_filter.filter(regex=r"adr_|com_")

,adr_num,adr_voie,com_cp,com_nom
0,,avenue albert Einstein,34000,Montpellier
1,,,0,
2,,,0,
3,,,0,
4,155,Rue de Bologne,34172,Montpellier
...,...,...,...,...
270,,avenue agropolis,34090,Montpellier
271,-,Avenue du Biterrois,34080,Montpellier
272,29,boulevard Sarrail,34000,Montpellier
273,"1 place Jacques Mirouse, MONTPELLIER","1 place Jacques Mirouse, MONTPELLIER",34000,Montpellier


#### adr_num field

In [9]:
data_filter.adr_num

0                                          
1                                          
2                                          
3                                          
4                                       155
                       ...                 
270                                        
271                                       -
272                                      29
273    1 place Jacques Mirouse, MONTPELLIER
274                                     105
Name: adr_num, Length: 275, dtype: object

#### adr_voie field

In [10]:
data_filter.adr_voie

0                    avenue albert Einstein
1                                          
2                                          
3                                          
4                            Rue de Bologne
                       ...                 
270                        avenue agropolis
271                     Avenue du Biterrois
272                       boulevard Sarrail
273    1 place Jacques Mirouse, MONTPELLIER
274                      rue Gustave Eiffel
Name: adr_voie, Length: 275, dtype: object

#### com_cp field

In [11]:
data_filter.com_cp

0      34000
1          0
2          0
3          0
4      34172
       ...  
270    34090
271    34080
272    34000
273    34000
274    34070
Name: com_cp, Length: 275, dtype: int64

#### com_nom field

In [12]:
data_filter.com_nom

0      Montpellier
1                 
2                 
3                 
4      Montpellier
          ...     
270    Montpellier
271    Montpellier
272    Montpellier
273    Montpellier
274    Montpellier
Name: com_nom, Length: 275, dtype: object

### Contact info

In [13]:
data_filter.tel1

0       334 67 64 87 70
1                      
2                      
3                      
4      +334 67 40 33 57
             ...       
270     334 67 54 45 20
271     334 67 75 44 43
272     334 67 34 71 33
273                    
274     334 67 75 10 55
Name: tel1, Length: 275, dtype: object

### Latest maintenance date

date_filter.dermnt

### Latitude and longitude

In [14]:
data_filter.filter(regex=r"_coor")

,lat_coor1,long_coor1
0,3.93392108647369,43.6136351580956
1,,
2,,
3,,
4,,
...,...,...
270,3.87346840734586,43.6409437259243
271,3.82081921967322,43.6327880433471
272,3.88032026773112,43.6106902122358
273,3.86982233698134,43.6127835439949


## Review selected cases and save as sample dirty data

In [15]:
sample_dirty = data_filter.loc[list(idx_problem_cases)]
sample_dirty

,nom,adr_num,adr_voie,com_cp,com_nom,tel1,freq_mnt,dermnt,lat_coor1,long_coor1


In [ ]:
sample_dirty.to_csv('data/sample_dirty.csv')